There is plethora of Postgis Raster functions for exporting the raster (https://postgis.net/docs/RT_reference.html#Raster_Outputs):

- ST_AsBinary/ST_AsWKB — Return the Well-Known Binary (WKB) representation of the raster.

- ST_AsHexWKB — Return the Well-Known Binary (WKB) in Hex representation of the raster.

- ST_AsGDALRaster — Return the raster tile in the designated GDAL Raster format. Raster formats are one of those supported by your compiled library. Use ST_GDALDrivers() to get a list of formats supported by your library.

- ST_AsJPEG — Return the raster tile selected bands as a single Joint Photographic Exports Group (JPEG) image (byte array). If no band is specified and 1 or more than 3 bands, then only the first band is used. If only 3 bands then all 3 bands are used and mapped to RGB.

- ST_AsPNG — Return the raster tile selected bands as a single portable network graphics (PNG) image (byte array). If 1, 3, or 4 bands in raster and no bands are specified, then all bands are used. If more 2 or more than 4 bands and no bands specified, then only band 1 is used. Bands are mapped to RGB or RGBA space.

- ST_AsTIFF — Return the raster selected bands as a single TIFF image (byte array). If no band is specified or any of specified bands does not exist in the raster, then will try to use all bands.

It's worth noting that these function just output the raster in the specified format - they do not save it to the disk. We can save the raster to disk using `gdal_translate`.


## Export with GDAL

In [5]:
!gdal_translate -co COMPRESS=DEFLATE -co ZLEVEL=9 PG:"host=localhost port=5432 dbname=giws user=user password=user schema=public table=dtm mode=2" dtm.tiff

Warning 1: Cannot find (valid) information about public.dtm table in raster_columns view. The raster table load would take a lot of time. Please, execute AddRasterConstraints PostGIS function to register this table as raster table in raster_columns view. This will save a lot of time.
Input file size is 12, 8
0...10...20...30...40...50...60...70...80...90...100 - done.


We use mode=2 option if we export a tiled raster: https://gdal.org/drivers/raster/postgisraster.html

In [2]:
%run sql_magics.ipynb

In [8]:
%%sql
------
-- you can get this warning when exporting:
--    Warning 1: Cannot find (valid) information about public.dtm table in raster_columns view. The raster table load would take a lot of time. Please, execute AddRasterConstraints PostGIS function to register this table as raster table in raster_columns view. This will save a lot of time.
------
SELECT AddRasterConstraints('dtm'::name, 'rast'::name);

NOTICE:  Adding SRID constraint
NOTICE:  The constraint "enforce_srid_rast" already exists.  To replace the existing constraint, delete the constraint and call ApplyRasterConstraints again
NOTICE:  Adding scale-X constraint
NOTICE:  The constraint "enforce_scalex_rast" already exists.  To replace the existing constraint, delete the constraint and call ApplyRasterConstraints again
NOTICE:  Adding scale-Y constraint
NOTICE:  The constraint "enforce_scaley_rast" already exists.  To replace the existing constraint, delete the constraint and call ApplyRasterConstraints again
NOTICE:  Adding blocksize-X constraint
NOTICE:  The constraint "enforce_width_rast" already exists.  To replace the existing constraint, delete the constraint and call ApplyRasterConstraints again
NOTICE:  Adding blocksize-Y constraint
NOTICE:  The constraint "enforce_height_rast" already exists.  To replace the existing constraint, delete the constraint and call ApplyRasterConstraints again
NOTICE:  Adding alignment co

Another option for export is from the SQL, but this is considered a bit of a hack
-- https://postgis.net/docs/using_raster_dataman.html#RasterOutput_PSQL

There are also several other options of export involving high level programming languages connecting to database and using raster output functions (https://postgis.net/docs/using_raster_dataman.html#RT_Raster_Applications)

Last but not least, raster data can be published as a web service:
- with MapServer https://postgis.net/docs/RT_FAQ.html#idm29185
- https://github.com/lcalisto/workshop-postgis-raster/blob/master/doc/geoserver.md